In [2]:
import logging

In [3]:
class MetricsClient:
    """
    타사 지표 전송 클라이언트
    """
    def send(self, metric_name, metric_value):
        if not isinstance(metric_name,str):
            raise TypeError("metric_name으로 문자열 타입을 사용해야 함")
        if not isinstance(metric_value,str):
            raise TypeError("metric_value로 문자열 타입을 사용해야 함")

        logging.info("%s 전송 값 = %s", metric_name, metric_value)

In [4]:
class Process:
    def __init__(self):
        self.client = MetricsClient() # 타사 지표 전송 클라이언트

    def process_interations(self, n_iterations):
        for i in range(n_iterations):
            result = self.run_process()
            self.client.send("iteration.".format(i), result)

- 메서드가 작다는 것은 테스트 가능성과 관련해 좋은 디자인이라고 할 수 있다.
- 마지막으로 필요한 부분만 테스트하기 위해 main 메서드에서 client를 직접 다루지 않고 래퍼(wrapper)메서드에 위임한다.

In [5]:
class WrappedClient:
    def __init__(self):
        self.client = MetricsClient()
    
    def send(self, metric_name, metric_value):
        return self.client.send(str(metric_name), str(metric_value))

In [6]:
class Process:
    def __init__(self):
        self.client = WrappedClient()

    def process_interations(self, n_iterations):
        for i in range(n_iterations):
            result = self.run_process()
            self.client.send("iteration.".format(i), result)
        

In [7]:
import unittest
from unittest.mock import Mock

In [8]:
class TestWrappedClient(unittest.TestCase):
    def test_send_converts_types(self):
        wrapped_client = WrappedClient()
        wrapped_client.client = Mock()
        wrapped_client.send("value", 1)
        wrapped_client.client.send.assert_called_with("value", "1")

## unittest

- 한명 이상의 사용자가 변경 내용에 동의하지 않은 경우 머지 리퀘스트가 거절(reject)된다.
- 아무도 반대하지 않은 상태에서 두 명 이상의 개발자가 동의하면 해당 머지 리퀘스트는 승인(approved)된다.
- 이외의 상태는 보류(pending) 상태이다.

In [18]:
from enum import Enum

class MergeRequestStatus(Enum):
    APPROVED = "approved"
    REJECTED = "rejected"
    PENDING = "pending"

class MergeRequest:
    def __init__(self):
        self._context = {
            "upvotes":set(),
            "downvotes":set()
        }
        self._status = MergeRequestStatus.OPEN
    
    def close(self):
        self._status = MergeRequestStatus.CLOSED
    
    @property
    def status(self):
        if self._context["downvotes"]:
            return MergeRequestStatus.REJECTED
        
        elif len(self._context["upvotes"]) >=2:
            return MergeRequestStatus.APPROVED
            return MergeRequestStatus.PENDING
        
    def upvote(self, by_user):
        self._cannot_vote_if_closed()

        self._context["downvotes"].discard(by_user)
        self._context["upvotes"].add(by_user)

    def downvote(self, by_user):
        self._cannot_vote_if_closed()

        self._context["upvotes"].discard(by_user)
        self._context["downvotes"].add(by_user)

    def _cannot_vote_if_closed(self):
        if self._status == MergeRequestStatus.CLOSED:
            raise MergeRequestException("종료된 머지 리퀘스트에 투표할 수 없음")




In [10]:
import unittest

In [11]:
class TestMergeRequestStatus(unittest.TestCase):
    def test_simple_rejected(self):
        merge_request = MergeRequest()
        merge_request.downvote("maintainer")
        self,assertEqual(merge_request.status, MergeRequestStatus.REJECTED)

    def test_just_created_is_pending(self):
        self.assertEqual(MergeRequest().status, MergeRequestStatus.PENDING)

    def test_pendin_awaiting_review(self):
        merge_request = MergeRequest()
        merge_request.upvode("core-dev")
        self.assertEqual(merge_request.status, MergeRequestStatus.PENDING)

    def test_approved(self):
        merge_request = MergeRequest()
        merge_request.upvote("dev1")
        merge_request.upvote("dev2")
        self.assertEqual(merge_request.status, MergeRequestStatus.APPROVED)

    def test_cannot_upvote_on_closed_merge_request(self):
        self.merge_request.close()
        self.assertRaises(
            MergeRequestException, self.merge_request.upvote, "dev1"
        )

    def test_cannot_downvote_on_closed_merge_request(self):
        self.merge_request.close()
        self.assertRaisesRegex(
            "종료된 머지 리퀘스트에 투표할 수 없음",
            self.merge_request.downvote,
            "dev1"
        )

## pytest

In [12]:
pip install pytest

Note: you may need to restart the kernel to use updated packages.


In [14]:
import pytest

In [15]:
def test_just_created_is_pending():
    assert MergeRequest().status == MergeRequestStatus.PENDING

- 간단히 결과가 참인지를 비교하는 것은 assert 구문만 사용되지만 예외의 발생 유무 검사와 같은 검사는 일부 함수를 사용해야한다.

In [20]:
def test_invalid_types():
    merge_request = MergeRequest()
    pytest.raises(TypeError, merge_request.upvote, {"invalid-object"})

def test_cannot_vote_on_closed_merge_request():
    merge_request = MergeRequest()
    merge_request.close()
    pytest.raises(MergeRequestException, merge_request.upvote, "dev1")
    with pytest.raises(MergeRequestException,
        match="종료된 머지 리퀘스트에 투표할 수 없음",):
            merge_request.downvote("dev1")

## 리팩토링

In [ ]:
from datetime import datetime

from a_module import STATUS_ENDPOINT

class BuildStatus:
    endpoint = STATUS_ENDPOINT

    def __init__(self, transport):
        self.transport = transport

    @staticmethod
    def build_date() -> str:
        return datetime.utcnow().isoformat()

    def compose_payload(self, merge_request_id, status) -> dict:
        return {
            "id": merge_request_id,
            "status": status,
            "built_at": self.build_date()
        }

    def deliver(self, payload):
        response = self.transport.post(self.endpoint, json=payload)
        response.raise_for_status()
        return response
    
    def notify(self, merge_request_id, status):
        return self.deliver(self.compose_payload(merge_request_id, status))

- notify를 분리하여 compose와 deliver로 나누고, 클래스를 패치하지 않고 바꿀 수 있도록 compose_payload()라는 새로운 메서드를 만들고, transport라는 의존성을 주입한다.

In [ ]:
@pytest.fixture
def build_status():
    bstatus = BuildSTATUS(Mock())
    bstatus.build_date = Mock(return_value="2018-01-01T00:00:01")
    return bstatus

def test_build_notifivation_sent(build_status):
    build_status.notify(1234,"OK")

    expected_payload = {
        "id": 1234,
        "status": "OK",
        "built_at": build_status.build_date(),
    }

    build_status.transport.post.assert_called_with(build_status.endpoint, json=expected_payload)